In [ ]:
!git clone https://github.com/MeowwMeoww/BeautyMomentSynthesis.git

In [ ]:
%cd /content/BeautyMomentSynthesis
!pip install -r requirements.txt

In [ ]:
!ngrok authtoken token

In [ ]:
from flask import Flask, request
from flask_ngrok import run_with_ngrok
from contextlib import contextmanager
import sys, os
import shutil
import json
from flask_cors import CORS, cross_origin
import re
import zipfile
import gdown

app = Flask(__name__)
cors = CORS(app)
app.config['CORS_HEADERS'] = 'Content-Type'


def unzip_and_save(url, purpose):
  os.makedirs('/content/zip_files', exist_ok = True)
  gg_drive_flag = 'drive.google.com' in url 

  if gg_drive_flag: #if True, the link is a shareable link from gg drive
    url = re.findall('[-\w]{25,}', url)[0]
    url = 'https://drive.google.com/uc?id=' + url

  if purpose == 'anchor':
    os.makedirs('/content/images/anchor_images', exist_ok = True)
    gdown.download(url , '/content/zip_files/anchor.zip', quiet = True)
    with zipfile.ZipFile('/content/zip_files/anchor.zip', 'r') as zip_ref:
      zip_ref.extractall('/content/images/anchor_images')

  elif purpose == 'input':
    os.makedirs('/content/images/original_images', exist_ok = True)
    gdown.download(url , '/content/zip_files/original.zip', quiet = True)
    with zipfile.ZipFile('/content/zip_files/original.zip', 'r') as zip_ref:
      zip_ref.extractall('/content/images/original_images')


@contextmanager
def suppress_stdout():
    with open(os.devnull, "w") as devnull:
        old_stdout = sys.stdout
        sys.stdout = devnull
        try:  
            yield
        finally:
            sys.stdout = old_stdout


def running_program(anchor_url = '/content/images/anchor_images', input_url = '/content/images/original_images', people_names = None):
  %cd /content/BeautyMomentSynthesis
  with suppress_stdout():
    if people_names != '':
      !python 'main.py' --anchor_dataset_path '{anchor_url}' --original_dataset_path '{input_url}' --output_path "/content/drive/MyDrive/Demo Beauty Moment Synthesis/final_video.avi" --auto_vid_params True --find_person people_names --number_of_images 20 
    else:
      !python 'main.py' --anchor_dataset_path '{anchor_url}' --original_dataset_path '{input_url}' --output_path "/content/drive/MyDrive/Demo Beauty Moment Synthesis/final_video.avi" --auto_vid_params True  --number_of_images 20 


def clear_program(img_url = '/content/images', zip_url = '/content/zip_files'):
  if os.path.exists(img_url) and os.path.exists(zip_url):
    shutil.rmtree(img_url)
    shutil.rmtree(zip_url)


@app.route('/', methods = ['POST', 'GET'])
@cross_origin()
def get_result():
  %cd /content
  data = request.get_json(force = True)
  data = json.loads(data)
  anchor_provided_url = data["anchor_path"]
  input_provided_url = data["input_path"]
  people_names = data["people_names"]
  unzip_and_save(anchor_provided_url, 'anchor')
  unzip_and_save(input_provided_url, 'input')
  running_program(people_names)
  clear_program()

  dict_result = {'video_path' : 'https://drive.google.com/file/d/1mJDe6l_M5Uzjk4quoRDi4dI5VMbcGeCl/view?usp=sharing'}

  return json.dumps(dict_result)
 

run_with_ngrok(app)

app.run()